In [ ]:
''''


algorith of id3:

ID3 (Examples, Target_Attribute, Attributes)
    Create a root node for the tree
    If all examples are positive, Return the single-node tree Root, with label = +.
    If all examples are negative, Return the single-node tree Root, with label = -.
    If number of predicting attributes is empty, then Return the single node tree Root,
    with label = most common value of the target attribute in the examples.
    Otherwise Begin
        A ← The Attribute that best classifies examples.
        Decision Tree attribute for Root = A.
        For each possible value, vi, of A,
            Add a new tree branch below Root, corresponding to the test A = vi.
            Let Examples(vi) be the subset of examples that have the value vi for A
            If Examples(vi) is empty
                Then below this new branch add a leaf node with label = most common target value in the examples
            Else below this new branch add the subtree ID3 (Examples(vi), Target_Attribute, Attributes – {A})
    End
    Return Root
...............................................................
the id3 algorithm is used to create decision tree from the given dataset.the result of tree is used to classify futuer sample 
data which use the notation of information gain that value is related to entropy
lets consider the p is the positive n is the negative.if p and n value is equel then it have highest entropy (1),if any value 
is zero then it have lowest entropy(0),

we want to find information gain from the attribute ,for that
    IG(Attribute) = Entropy of attribute — Weighted average of Entropy of each child set
this equcation is used
..........................................
*first the entropy of the total dtaset is calculated
*the dataset is then split on the different attributes
*the entyropy for each branche is calculated then it added proportionaly,to get total entropy for the split
*the resulting entropy is substracted from the entropy before the split
*the result is the IG,or decresing the entropy
*the attribute that yields the largest IG is choose for contruct the node
...........................
but by using sklearn we can directly create the decision tree DecisionTreeClassifier()is the building function used to 
create decision tree
................................
advantage:  build a short tree
            builds the fastest tree
            understandable prediction rule
            find the leaf node enables test data to be pruned reduceing number of test
            tree is created based on all dataset
disadvantage: only one attribute at a time is tested for making decision
              classifying continuous data may be computationaly expensive
              data may be overfitted over classified if a small sample is tested


In [1]:
#importing libraries
import os
import pandas as pd
import numpy as np

In [2]:
#open the dataset files and list the directories available in that dataset file
path="C:\\Users\\best\\Downloads\\Earthquake_damage\\Earthquake_damage"
print(os.listdir(path))

['Building_Ownership_Use.csv', 'Building_Structure.csv', 'ReadMe.md', 'sample_submission.csv', 'test.csv', 'train.csv']


In [3]:
#open the csv file ,take first 10000 data from total data.pd.read_csv is used to read the csv file and do operations on it
training=pd.read_csv(path+'\\train.csv')      
training.head()
data1=training[:10000]

In [4]:
data2=pd.read_csv(path+'\\Building_Ownership_Use.csv')
data2.head()
data3 = pd.merge(data2,data1,on=['building_id'])

In [5]:
data4=pd.read_csv(path+'\\Building_Structure.csv')
data4.head()
data= pd.merge(data4,data3,on=['building_id'])

In [6]:
data.head()

,building_id,district_id,vdcmun_id,ward_id_x,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,...,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,vdcmun_id_y
0,6627e91213e,7,702,70201,1,1,25,324,8,8,...,0.0,0,0,0,0,0,0,0,1.0,702
1,a3773aa267,7,702,70208,3,3,22,458,24,24,...,0.0,0,0,0,0,0,0,0,1.0,702
2,a3b10bc634,7,703,70305,1,2,4,600,10,20,...,0.0,0,0,0,0,0,0,0,1.0,703
3,a3b1a442ee,7,703,70306,2,2,27,425,14,14,...,1.0,0,0,1,0,0,0,0,1.0,703
4,a3b1a44cb2,7,703,70306,2,2,15,425,14,14,...,0.0,0,0,0,0,0,0,0,1.0,703


In [7]:
print(len(data))

10000


In [8]:
#list the what are the featuers consider for training
features=list(data)
print(features)

['building_id', 'district_id', 'vdcmun_id', 'ward_id_x', 'count_floors_pre_eq', 'count_floors_post_eq', 'age_building', 'plinth_area_sq_ft', 'height_ft_pre_eq', 'height_ft_post_eq', 'land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered', 'has_superstructure_other', 'condition_post_eq', 'district_id_x', 'vdcmun_id_x', 'ward_id_y', 'legal_ownership_status', 'count_families', 'has_secondary_use', 'has_secondary_use_agriculture', 'has_secondary_use_hotel', 'has_secondary_use_rental', 'has_secondary_use_institution', 'has_secondary_use_school', 'has_secondary_u

In [9]:
#creating a dictionary for storing the  numeric value of corresponding string 
dictionary={}
i=0

In [10]:
#convertion is a function used to convert the string values in to numeric value ,where we take word is the key,
def convertion(dictionary,word):
    if word not in dictionary:
        value=len(dictionary)
        dictionary[word]=value
        return dictionary[word]
    else:
        return dictionary[word]
        

In [11]:
#go through each cell ,that cell contain value stored into variable word.then find the datatype of each cell values ,
#if it is objective then call convertion function.then we get the neumeric value corresponding object.then it consider the new
#new value of the that perticular cell
i=0
for i in range(len(data)):
    for feature in features:
        word=data.loc[i,feature]
        if data[feature].dtype==object:
            convertion(dictionary,word)
            new_data=dictionary[word]
            data.loc[i,feature]=new_data

In [12]:
print(len(data))

10000


In [13]:
#from the dataset any cell is empty then it replace with 0
data.fillna(0,inplace=True)

In [14]:
#we create two class with or without grade.so that in x contain dataset without grade and y contain with label.for 
#training purpose we make this category
x = data.drop('damage_grade', axis=1)
y = data['damage_grade']



In [15]:
#we want training and testing dataset for that we splite the available dataset by using train_test_split.spliting done with 80% training 20% for testing
from sklearn.model_selection import train_test_split
x_Train, x_Test, y_Train, y_Test = train_test_split(x, y, test_size = 0.2,train_size=0.8)

In [16]:
x_Train.head()

,building_id,district_id,vdcmun_id,ward_id_x,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,...,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,vdcmun_id_y
5809,5860,24,2406,240610,3,3,10,1220,27,27,...,0.0,0,0,0,0,0,0,0,0.0,2406
4037,4087,21,2127,212702,2,2,25,135,18,18,...,0.0,0,0,0,0,0,0,0,0.0,2127
7380,7431,28,2860,286006,1,1,15,351,12,12,...,0.0,0,0,0,0,0,0,0,0.0,2860
3521,3571,20,2030,203008,2,2,9,120,8,8,...,1.0,0,0,1,0,0,0,0,1.0,2030
312,354,11,1129,112904,3,3,21,555,21,21,...,0.0,0,0,0,0,0,0,0,1.0,1129


In [17]:
x_Test.head()

,building_id,district_id,vdcmun_id,ward_id_x,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,...,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,vdcmun_id_y
2527,2576,25,2511,251109,2,0,34,251,17,0,...,0.0,0,0,0,0,0,0,0,0.0,2511
8707,8758,31,3119,311901,2,2,5,200,10,10,...,0.0,0,0,0,0,0,0,0,1.0,3119
6478,6529,24,2472,247202,2,2,22,405,15,15,...,0.0,0,0,0,0,0,0,0,0.0,2472
1037,1084,38,3831,383104,1,2,13,176,10,10,...,0.0,0,0,0,0,0,0,0,0.0,3831
3414,3464,20,2025,202506,1,1,1,500,10,10,...,0.0,0,0,0,0,0,0,0,0.0,2025


In [18]:
y_Train.head()

5809    28
4037    28
7380    11
3521    20
312     11
Name: damage_grade, dtype: int64

In [19]:
y_Test.head()

2527    41
8707    28
6478    23
1037    20
3414    28
Name: damage_grade, dtype: int64

In [20]:
#create the decision tree object using DecisionTreeClassifier
from sklearn import tree

model = tree.DecisionTreeClassifier()

In [21]:
model

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [22]:
#Fitting the model to (i.e. using the .fit() method on) the training data is essentially the training part of the modeling process
model=model.fit(x_Train, y_Train)

In [23]:
#classify incoming data points (from a test set, or otherwise) using the predict
y_pred = model.predict(x_Test)

In [24]:
#At this point we have trained our algorithm and made some predictions. Now we'll see how accurate our algorithm is. 
#For classification tasks some commonly used metrics are confusion matrix, precision, recall, and F1 score. 
#Lucky for us Scikit=-Learn's metrics library contains the classification_report and confusion_matrix methods that can be used 
#to calculate these metrics for us:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_Test, y_pred))
print(classification_report(y_Test, y_pred))

[[146  83 129   7   3]
 [113  99  61  15   3]
 [133  55 258   4  22]
 [ 12  20   4 179   2]
 [  8   1  31   0 612]]
              precision    recall  f1-score   support

          11       0.35      0.40      0.37       368
          20       0.38      0.34      0.36       291
          23       0.53      0.55      0.54       472
          28       0.87      0.82      0.85       217
          41       0.95      0.94      0.95       652

    accuracy                           0.65      2000
   macro avg       0.62      0.61      0.61      2000
weighted avg       0.65      0.65      0.65      2000

